# SpecTrain Preprocessing
Review the data to be preprocessed.
There are two types of input data:
1. Clinical features (obtained by current routine methods)
2. NMR features (the spectra file and the measured outputs)


In [8]:
import os
import io

import gcsfs
from google.cloud import bigquery, storage


import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import (
    GRU,
    LSTM,
    RNN,
    Bidirectional,
    Conv1D,
    Dense,
    MaxPool1D,
    Reshape,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from scipy.signal import find_peaks

# To plot pretty figures
%matplotlib inline
mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

# For reproducible results.
from numpy.random import seed

seed(1)
tf.random.set_seed(2)

In [2]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
%env PROJECT = {PROJECT}
%env BUCKET = spectrain
%env REGION = "us-central1"

env: PROJECT=qwiklabs-asl-00-c812c3b423f2
env: BUCKET=spectrain
env: REGION="us-central1"


In [9]:
df=pd.read_csv('gs://qwiklabs-asl-00-c812c3b423f2/spec_train_output/input/Kidney_TX_data.csv')
df = df.filter(regex=r'^(?!LS|Banff|Biopsy|Source|Patient.S|Nmr)')
df.head()

,Patient.ID,Patient.Age.at.Biopsy,Patient.Age.at.TX,Case,Sex,serum_creatinine,hippurate,phenylacetylglutamine,trigonellin,urea,alanine,citrate,dimethylamine,lactate,Diabetes,Hypertension,UA.Pro,UA.Hb,Spectrum_file
0,bhdLeIiEnep6TPu8qeostZ8o(,55,55,0,male,2.15,0.027913,0.039593,0.012358,3.036788,1.038175,0.040761,0.232653,8.653069,False,False,NaN,NaN,output_NormalizationTool_spectrum_zgpr30-urine...
1,8agYxL3U(2m0pcw^vomp*Yu9x,56,55,0,female,0.90,0.012763,0.113517,0.040703,5.663656,1.025761,0.105713,0.316157,6.109724,False,True,False,False,output_NormalizationTool_spectrum_zgpr30-urine...
2,l849CAs#0wR1i(EqkyLtIxwZS,58,58,0,male,1.20,0.122959,0.283221,0.172492,7.417329,1.070371,0.231169,0.262318,17.866997,False,False,False,False,output_NormalizationTool_spectrum_zgpr30-urine...
3,lw*R@N7LoSHUuxTIxTs$PWzfg,47,45,0,male,1.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,output_NormalizationTool_spectrum_zgpr30-urine...
4,4oAgnWhMrp$h@B1*Um*PYowCS,22,22,1,female,1.67,0.018280,0.041485,0.010831,3.556433,1.029425,0.010481,0.356004,6.239394,False,False,True,False,output_NormalizationTool_spectrum_zgpr30-urine...


## Standard Clinical Features to Engineer
Initial modeling found serum creatinine, urine protein, age and sex to be useful factors.
- Protein is a logical
- Serum creatinine should be crossed with age and sex using the CKD-EPI equation
- Time since transplant should be calculated based on the difference between the age at biopsy and age at transplant
- Time since transplant could be bucketed into <1yr, 1 to 3yr and >3yr
- eGFR should be bucketed at <60 vs. >60

In [10]:
# Define the CKD-EPI equation function
def calculate_eGFR(row):
    if row['Sex'] == 'male':
        kappa = 0.9
        alpha = -0.302
        beta = 1.0
    else:
        kappa = 0.7
        alpha = -0.241
        beta = 1.012

    eGFR = 142 * min(row['serum_creatinine'] / kappa, 1)**alpha * \
           max(row['serum_creatinine'] / kappa, 1)**(-1.2) * \
           0.9938**row['Patient.Age.at.Biopsy'] * beta
    return eGFR

# Apply the function to create the 'eGFR' column
df['eGFR'] = df.apply(calculate_eGFR, axis=1)

#Calculate the time since transplant
def calculate_time(row):
    return row['Patient.Age.at.Biopsy'] - row['Patient.Age.at.TX']

df['time.TX'] = df.apply(calculate_time, axis=1)

def find_peaks_for_files(directory, df):
    # List to store the peak values
    peaks_list = []

    # Iterate over the files in the directory
    for filename in df['Spectrum_file']:
        file_path = os.path.join(directory, filename)
        
        # Read the .txt file into a DataFrame
        file_df = pd.read_csv(file_path, sep='\s', header=None)
        file_df.columns = ['x_axis', 'y_axis']
        
        # Filter the DataFrame for x_axis values between 0.1 and 9.3
        filtered_df = file_df[(file_df['x_axis'] >= 0.1) & (file_df['x_axis'] <= 9.3)]
        
        # Find the peaks in the filtered y_axis column
        peaks, _ = find_peaks(filtered_df['y_axis'])
        peak_values = filtered_df.loc[peaks, ['x_axis', 'y_axis']]
        
        # Append the peak values to the list
        peaks_list.append(peak_values)

    # Add the peaks_list to the df DataFrame
    df['peaks'] = peaks_list
    return df
DIRECTORY = 'gs://qwiklabs-asl-00-c812c3b423f2/Kidney_TX_Data'
find_peaks_for_files(DIRECTORY, df)

df.describe()

FileNotFoundError: [Errno 2] No such file or directory: 'Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_14_739_00000_withoutBackground_20001.txt'

In [5]:
def get_blob(blobs):
    for blob in blobs:
        yield blob

def write_png_to_gcs(blob, bucket):
    print(blob.name)
    #NOTE: important to use pyplot instantiation this way to ensure no memory leaks
    fig = plt.figure(num=1,figsize=(480,48), clear=True)
    data = blob.download_as_bytes()
    df = pd.read_csv(io.BytesIO(data), sep='\s', header=None)
    # df=pd.read_csv(blob.name, sep='\s', header=None)
    # filename=filename.split('.')[0]
    df.columns=['x_axis', 'y_axis']
    df = df[(df['x_axis'] >= 0.1) & (df['x_axis'] <= 9.3)]
    ax = fig.add_subplot()
    ax.plot(df['x_axis'], df['y_axis'])
    ax.axis('off')

    buf = io.BytesIO()
    fig.savefig(buf, format='png')

    filename = blob.name.split('.')[0] # remove the suffix/file extension
    filename = filename.split('/')[1] # remove the containing directory name from filename

    upload_blob = bucket.blob(image_dir+filename+'_nmr.png')
    upload_blob.upload_from_file(buf, content_type='image/png', rewind=True)

    # plt.close()
    buf.close()
    del df
    # del plt
    del data
    del upload_blob
    
    return

In [ ]:
image_dir='spec_train_output/processed_images/'
# plt.figure(figsize=(300,40))

# initialize the GCS client
storage_client = storage.Client()

# get the storage bucket
bucket = storage_client.get_bucket('spectrain')

# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs('spectrain', prefix='Kidney_TX_Data')

i = 0 # counter to use for breaking

# Note: The call returns a response only when the iterator is consumed.
for blob in get_blob(blobs):
    if("output" in blob.name):
        write_png_to_gcs(blob, bucket)
        #i = i+1
        #if(i == 3):
         #    break;


Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_571_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_572_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_573_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_574_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_575_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_576_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_577_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_578_00000_withoutBackground_20001.txt
Kidney_TX_Data/output_NormalizationTool_spectrum_zgpr30-urine-600MHz-310K_12_579

In [181]:
# Create data split column for images data import file
data_split_df=pd.read_csv('gs://spectrain/Kidney_TX_data_with_split.csv')
image_paths_df=pd.read_csv('gs://spectrain/spec_train_output/image_dir_paths_labels1.csv', header=None)
data_split_df['Spectrum_file_new'] = data_split_df.Spectrum_file.str.split('\.').str[0]
data_split_df['Spectrum_file_new'] = "gs://spectrain/spec_train_output/images/" + data_split_df['Spectrum_file_new'] + '_nmr.png'
data_split_df = data_split_df[['Spectrum_file_new', 'data_split']]
image_paths_df.columns=['Spectrum_file_new', 'Case']
image_paths_df=pd.merge(image_paths_df, data_split_df, on=['Spectrum_file_new'], how='inner')
image_paths_df=image_paths_df[['data_split', 'Spectrum_file_new', 'Case']]
image_paths_df['data_split'] = image_paths_df.data_split.replace({'TRAIN':'TRAINING', 'VALID':'VALIDATION', 'TEST':'TEST'})
image_paths_df.to_csv('gs://spectrain/spec_train_output/image_dir_paths_labels_with_split.csv', header=None, index=False)